# Определение возраста покупателей

## Исследовательский анализ данных

In [2]:
import tqdm
import matplotlib.pyplot as plt
import numpy as np


from tensorflow.keras.models import Sequential
import pandas as pd


from tensorflow.keras.datasets import fashion_mnist
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import Conv2D, Flatten, Dense, AveragePooling2D, AvgPool2D,GlobalAveragePooling2D

from tensorflow.keras.applications.resnet import ResNet50


from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing.image import ImageDataGenerator




labels = pd.read_csv('/datasets/faces/labels.csv')
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)
train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory='/datasets/faces/final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        seed=12345) 



In [ ]:
print(type(labels))

In [ ]:
display(labels.head())

In [ ]:
labels.info()

In [ ]:
labels['real_age'].hist()

print('Медиана:' , labels['real_age'].median())
print('Среднее:', labels['real_age'].mean())

In [ ]:
#fig = plt.figure(figsize=(10,10))

# labels['file_name'].head()
# print(len(train_gen_flow[0][0][0][0]))



## Краткий вывод

Данные смещены влево, справа виднеется хвост
Есть покупатели в возрасте 100 лет, вряд ли это выброс - это возможно.
Также присутствуют дети и средний возраст покупателя - 30 лет. 

In [ ]:
#выводим изображения

features, target = next(train_gen_flow)

fig = plt.figure(figsize=(10, 10))
for i in range(15):
    fig.add_subplot(4, 4, i+1)
    plt.imshow(features[i])
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()

### Краткий вывод 

Имеем 7591 подписанных фотографий, если не получится получить метрику MAE должного качества, то возможно придется заморозить ResNet и применить аугментацию. 

## Обучение модели

(Код в этом разделе запускается в отдельном GPU-тренажёре, поэтому оформлен отдельно)

In [ ]:
#Загрузка библиотек:
import matplotlib.pyplot as plt
import numpy as np


from tensorflow.keras.models import Sequential
import pandas as pd




from tensorflow.keras.layers import Conv2D, Flatten, Dense, AveragePooling2D, AvgPool2D,GlobalAveragePooling2D

from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


def load_train(path):
    labels = pd.read_csv(path + 'labels.csv')
    train_datagen = ImageDataGenerator(rescale=1/255, validation_split=0.25)
 
    train_datagen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path+"/final_files/",
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        subset='training',
        seed=12345)
 
    return train_datagen_flow
    
    
def load_test(path):
    
    labels = pd.read_csv(path + 'labels.csv')
    test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)
    
    test_datagen_flow = test_datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path+'/final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        subset='validation',
        seed=12345)
 
    return test_datagen_flow


def create_model(input_shape):

    
    backbone = ResNet50(input_shape=(224, 224, 3),
                    weights='/datasets/keras_models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', 
                    include_top=False)

    model = Sequential()
    
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    
    
    model.add(Dense(1, activation='relu'))
    
    
    
def train_model(model, train_data, test_data, batch_size=None, epochs=50,
               steps_per_epoch=None, validation_steps=None):
    

    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)
        
    
    model.fit(train_data,
              validation_data=test_data,
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2)

    return model





# Результат тестирования модели

```





2022-01-22 21:28:02.081838: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1241] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 30509 MB memory) -> physical GPU (device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:8b:00.0, compute capability: 7.0)
<class 'tensorflow.python.keras.engine.sequential.Sequential'>
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
Train for 178 steps, validate for 60 steps

178/178 - 44s - loss: 237.3509 - mae: 10.9347 - val_loss: 859.8865 - val_mae: 24.4461
Epoch 2/50
178/178 - 37s - loss: 63.2550 - mae: 5.9857 - val_loss: 617.2277 - val_mae: 19.6280
Epoch 3/50
178/178 - 36s - loss: 31.1667 - mae: 4.3168 - val_loss: 346.5691 - val_mae: 13.8039
Epoch 4/50
178/178 - 36s - loss: 19.0924 - mae: 3.3553 - val_loss: 122.8992 - val_mae: 8.3678
Epoch 5/50
178/178 - 36s - loss: 15.8479 - mae: 3.0802 - val_loss: 83.1254 - val_mae: 6.9366
Epoch 6/50
178/178 - 37s - loss: 12.9185 - mae: 2.7465 - val_loss: 72.7465 - val_mae: 6.6219
Epoch 7/50
178/178 - 36s - loss: 10.5581 - mae: 2.5017 - val_loss: 65.4750 - val_mae: 6.2292
Epoch 8/50
178/178 - 36s - loss: 9.7935 - mae: 2.3978 - val_loss: 68.6312 - val_mae: 6.2250
Epoch 9/50
178/178 - 36s - loss: 8.7853 - mae: 2.2377 - val_loss: 77.1705 - val_mae: 6.8465
Epoch 10/50
178/178 - 37s - loss: 7.2393 - mae: 2.0415 - val_loss: 69.1239 - val_mae: 6.1867
Epoch 11/50
178/178 - 37s - loss: 6.1940 - mae: 1.8611 - val_loss: 65.9357 - val_mae: 6.1934
Epoch 12/50
178/178 - 37s - loss: 5.6516 - mae: 1.7889 - val_loss: 60.4073 - val_mae: 5.8643
Epoch 13/50
178/178 - 36s - loss: 5.3803 - mae: 1.7531 - val_loss: 62.9613 - val_mae: 5.9623
Epoch 14/50
178/178 - 36s - loss: 5.4070 - mae: 1.7433 - val_loss: 64.9016 - val_mae: 6.0494
Epoch 15/50
178/178 - 37s - loss: 5.2132 - mae: 1.7077 - val_loss: 63.8899 - val_mae: 6.0815
Epoch 16/50
178/178 - 36s - loss: 4.7659 - mae: 1.6432 - val_loss: 74.9514 - val_mae: 6.4393
Epoch 17/50
178/178 - 36s - loss: 5.1575 - mae: 1.7019 - val_loss: 61.3842 - val_mae: 5.9628
Epoch 18/50
178/178 - 36s - loss: 5.3724 - mae: 1.7496 - val_loss: 79.2575 - val_mae: 6.6949
Epoch 19/50
178/178 - 37s - loss: 6.6018 - mae: 1.9065 - val_loss: 65.7615 - val_mae: 6.2671
Epoch 20/50
178/178 - 37s - loss: 6.9248 - mae: 1.9793 - val_loss: 71.5454 - val_mae: 6.6050
Epoch 21/50
178/178 - 36s - loss: 5.7461 - mae: 1.8398 - val_loss: 76.4248 - val_mae: 6.8256
Epoch 22/50
178/178 - 36s - loss: 4.8719 - mae: 1.6535 - val_loss: 73.9133 - val_mae: 6.4022
Epoch 23/50
178/178 - 36s - loss: 4.6954 - mae: 1.6047 - val_loss: 65.3461 - val_mae: 6.1257
Epoch 24/50
178/178 - 37s - loss: 4.3505 - mae: 1.5599 - val_loss: 72.4336 - val_mae: 6.3072
Epoch 25/50
178/178 - 36s - loss: 3.9863 - mae: 1.5044 - val_loss: 59.3708 - val_mae: 5.8373
Epoch 26/50
178/178 - 36s - loss: 3.7314 - mae: 1.4550 - val_loss: 58.1514 - val_mae: 5.7842
Epoch 27/50
178/178 - 36s - loss: 3.5391 - mae: 1.4190 - val_loss: 59.9471 - val_mae: 5.7805
Epoch 28/50
178/178 - 36s - loss: 3.6778 - mae: 1.4551 - val_loss: 68.3451 - val_mae: 6.1558
Epoch 29/50
178/178 - 36s - loss: 4.0138 - mae: 1.5030 - val_loss: 63.6000 - val_mae: 5.9179
Epoch 30/50
178/178 - 37s - loss: 4.2037 - mae: 1.5525 - val_loss: 60.5545 - val_mae: 5.8689
Epoch 31/50
178/178 - 36s - loss: 4.8169 - mae: 1.6611 - val_loss: 59.4502 - val_mae: 5.7248
Epoch 32/50
178/178 - 36s - loss: 4.7636 - mae: 1.6531 - val_loss: 59.7324 - val_mae: 5.8838
Epoch 33/50
178/178 - 37s - loss: 4.5070 - mae: 1.5910 - val_loss: 62.2998 - val_mae: 5.9643
Epoch 34/50
178/178 - 36s - loss: 4.1296 - mae: 1.5221 - val_loss: 61.7840 - val_mae: 5.9817
Epoch 35/50
178/178 - 36s - loss: 3.7143 - mae: 1.4487 - val_loss: 61.9931 - val_mae: 5.9583
Epoch 36/50
178/178 - 36s - loss: 3.3221 - mae: 1.3676 - val_loss: 64.4045 - val_mae: 6.1169
Epoch 37/50
178/178 - 37s - loss: 3.0789 - mae: 1.3217 - val_loss: 60.1383 - val_mae: 5.8132
Epoch 38/50
178/178 - 36s - loss: 2.7882 - mae: 1.2640 - val_loss: 58.7526 - val_mae: 5.8036
Epoch 39/50
Epoch 40/50
178/178 - 36s - loss: 2.7730 - mae: 1.2626 - val_loss: 60.0741 - val_mae: 5.8164
Epoch 41/50
178/178 - 36s - loss: 2.9736 - mae: 1.3037 - val_loss: 59.7366 - val_mae: 5.7657
178/178 - 36s - loss: 3.1545 - mae: 1.3392 - val_loss: 62.0124 - val_mae: 5.7905
Epoch 42/50
178/178 - 36s - loss: 3.3371 - mae: 1.3622 - val_loss: 62.8570 - val_mae: 5.8975
Epoch 43/50
178/178 - 36s - loss: 3.5649 - mae: 1.4216 - val_loss: 61.9474 - val_mae: 5.8941
Epoch 44/50
178/178 - 37s - loss: 3.7023 - mae: 1.4409 - val_loss: 61.0695 - val_mae: 5.8210
Epoch 45/50
178/178 - 37s - loss: 3.5208 - mae: 1.4049 - val_loss: 61.8819 - val_mae: 5.9617
Epoch 46/50
178/178 - 36s - loss: 3.4236 - mae: 1.3911 - val_loss: 68.7236 - val_mae: 6.1108
Epoch 47/50
178/178 - 37s - loss: 3.3571 - mae: 1.3715 - val_loss: 63.3206 - val_mae: 5.8728
Epoch 48/50
178/178 - 37s - loss: 3.0963 - mae: 1.2995 - val_loss: 61.7898 - val_mae: 5.9845
Epoch 49/50
178/178 - 36s - loss: 2.6959 - mae: 1.2283 - val_loss: 62.1521 - val_mae: 5.9943
Epoch 50/50
178/178 - 36s - loss: 2.4876 - mae: 1.1668 - val_loss: 58.4720 - val_mae: 5.7638
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']



## Анализ обученной модели

В ходе предварительного анализа были получены размеры входящих изображений (224,224)
В модели была реализована последовательная нейросеть с использованией предобученной сети ResNet50, без "верхушки", с одним слоем пулинга, усредняющего информацию с изображения, которая заканчивалась 1 нейроном (так как нам по заданию нужно получить одно число) , с функцией активации 'Relu', которая позволяет отбросить не допустить на выход отрицательные значения. В итоге за 50 эпох модель показала Среднюю абсолютною погрешность в 5,7.